In [40]:
import bs4
import requests
import pandas as pd
import re

In [41]:
urls = []
addresses = []
names = []
ids = []

for page_number in range(1, 51):
    URL = f"https://villahanem.com/kiralik-villalar?s={page_number}"
    response = requests.get(URL)
    response.encoding = "utf-8" 
    html_parsed = bs4.BeautifulSoup(response.text, "html.parser")
    
    urls.extend(["https://villahanem.com"+villa_url["href"] for villa_url in html_parsed.select("#searchResult .resultItem .img a")[::2]])
    addresses.extend([villa_loc.getText().strip() for villa_loc in html_parsed.select(".location.listHide")])
    villa_names = [villa_name.getText().strip() for villa_name in html_parsed.select("strong.title")][1:]
    names.extend(villa_names)
    ids.extend([villa_name.lower().replace(" ", "") for villa_name in villa_names])
    
    if page_number % 10 == 0:
        print(f"{page_number} pages have been checked!")
        
print(f"\n---OPERATION HAS BEEN COMPLETED!---")
print(f"{page_number} pages have been checked!")
print(f"Total Villa URL: {len(urls)}")

10 pages have been checked!
20 pages have been checked!
30 pages have been checked!
40 pages have been checked!
50 pages have been checked!

---OPERATION HAS BEEN COMPLETED!---
50 pages have been checked!
Total Villa URL: 596


In [42]:
guests = []
bedrooms = []
bathrooms = []
descriptions = []
latitudes = []
longitudes = []
photos = []
average_prices = []
q1_prices = []
q2_prices = []
q3_prices = []
q4_prices = []

URL_ERRORS = []
villa_number = 0
for URL in urls:
    try:
        response = requests.get(URL)
        response.encoding = "utf-8" 
        html_parsed = bs4.BeautifulSoup(response.text, "html.parser")
        villa_number += 1
    except:
        print(f"{URL} : URL ERROR!")
        URL_ERRORS = []
        
    try:
        prop_elems = html_parsed.select(".propertiesList.ml-auto li")
        guest = int(prop_elems[0].getText().replace(prop_elems[0].select_one("span").getText(), ""))
    except:
        print(f"{URL} : GUEST NUMBER ERROR!")
        guest = None
        
    try:
        prop_elems = html_parsed.select(".propertiesList.ml-auto li")
        bedroom = int(prop_elems[1].getText().replace(prop_elems[1].select_one("span").getText(), ""))
    except:
        print(f"{URL} : BEDROOM NUMBER ERROR!")
        bedroom = None
    
    try:
        prop_elems = html_parsed.select(".propertiesList.ml-auto li")
        bathroom = int(prop_elems[2].getText().replace(prop_elems[2].select_one("span").getText(), ""))
    except:
        print(f"{URL} : BATHROOM NUMBER ERROR!")
        bathroom = None
        
    try:
        description = ("\n".join([text_elem.getText().strip() for text_elem in html_parsed.select("#description p")])).strip()
    except:
        print(f"{URL} : DESCRIPTION ERROR!")
        description = None
        
    try:
        latitude = float(html_parsed.select_one("#detailMap")["data-lat"].strip())
        longitude = float(html_parsed.select_one("#detailMap")["data-lng"].strip()) 
    except:
        print(f"{URL} : LOCATION ERROR!")
        latitude = None
        longitude = None
        
    try:
        villa_photos = tuple("https://villahanem.com"+villa_photo["href"] for villa_photo in html_parsed.select(".photoGallery.detayGallery a"))        
    except:
        print(f"{URL} : PHOTO ERROR")
        villa_photos = None
        
    try:
        villa_prices = [int(re.sub("[^0-9]", "", price_elem.getText().strip())) for price_elem in html_parsed.select("#priceTable td")[3::4]]
        avg_price = sum(villa_prices) // len(villa_prices)
        q1_price = avg_price*4 * 0.14
        q2_price = avg_price*4 * 0.32
        q3_price = avg_price*4 * 0.37
        q4_price = avg_price*4 * 0.17
    except:
        print(f"{URL} : PRICE ERROR!")
        avg_price = None
        q1_price = None
        q2_price = None
        q3_price = None
        q4_price = None
    
    guests.append(guest)
    bedrooms.append(bedroom)
    bathrooms.append(bathroom)
    descriptions.append(description)
    latitudes.append(latitude)
    longitudes.append(longitude)
    average_prices.append(avg_price)
    q1_prices.append(q1_price)
    q2_prices.append(q2_price)
    q3_prices.append(q3_price)
    q4_prices.append(q4_price)
    photos.append(villa_photos)   
    
    if villa_number % 75 == 0:
        print(f"{villa_number} villas have been completed!")

print(f"THE OPERATION IS COMPLETED!")
print(f"TOTAL VILLA NUMBER: {villa_number}")

https://villahanem.com/Villa-Zumak-istanbul : LOCATION ERROR!
https://villahanem.com/Villa-Burak-Bey-Konagi : LOCATION ERROR!
https://villahanem.com/Villa-Ulya : LOCATION ERROR!
https://villahanem.com/Villa-Zeliha-Hanim-Konagi : LOCATION ERROR!
https://villahanem.com/Villa-Zelis : LOCATION ERROR!
https://villahanem.com/Villa-Ceyhun : LOCATION ERROR!
https://villahanem.com/Villa-Benk-Palas : LOCATION ERROR!
https://villahanem.com/Villa-Askhane : LOCATION ERROR!
https://villahanem.com/Villa-Su-Kalkan- : LOCATION ERROR!
https://villahanem.com/villa-lamazi-4 : LOCATION ERROR!
https://villahanem.com/Villa-Hira : LOCATION ERROR!
https://villahanem.com/Villa-Lamazi-House- : LOCATION ERROR!
https://villahanem.com/Villa-Voluptas : LOCATION ERROR!
https://villahanem.com/Villa-Mutlu-Hane : LOCATION ERROR!
https://villahanem.com/Villa-Arya : LOCATION ERROR!
https://villahanem.com/Villa-Bade-Duo : LOCATION ERROR!
https://villahanem.com/Villa-3-K : LOCATION ERROR!
https://villahanem.com/Villa-Yaren 

In [43]:
data_raw = {
    "ID": ids,
    "Name": names,
    "Address": addresses,
    "Guest Number": guests,
    "Bedroom Number": bedrooms,
    "Bathroom Number": bathrooms,
    "Latitude": latitudes,
    "Longitude": longitudes,
    "Description": descriptions,
    "Photo URLs": photos,
    "Average Price": average_prices,
    "Q1 Price": q1_prices,
    "Q2 Price": q2_prices,
    "Q3 Price": q3_prices,
    "Q4 Price": q4_prices
}

df_raw = pd.DataFrame(data_raw)
df_raw.set_index("ID", inplace=True)
df_raw.to_excel("villahanem_raw.xlsx")

data_info = {
    "ID": ids,
    "Name": names,
    "Address": addresses,
    "Guest Number": guests,
    "Bedroom Number": bedrooms,
    "Bathroom Number": bathrooms,
    "Latitude": latitudes,
    "Longitude": longitudes,
    "Description": descriptions,
    "Photo URLs": photos  
}

df_info = pd.DataFrame(data_info)
df_info.set_index("ID", inplace=True)
df_info.to_excel("villahanem_info.xlsx")

data_financial = {
    "ID": ids,
    "Average Price": average_prices,
    "Q1 Price": q1_prices,
    "Q2 Price": q2_prices,
    "Q3 Price": q3_prices,
    "Q4 Price": q4_prices   
}

df_financial = pd.DataFrame(data_financial)
df_financial.set_index("ID", inplace=True)
df_financial.to_excel("villahanem_financial.xlsx")